In [ ]:
from bs4 import BeautifulSoup
import urllib2
import logging
import requests
import re
import telegram
from time import sleep
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

try:
    from urllib.error import URLError
except ImportError:
    from urllib2 import URLError  # python 2

def main():
    # Telegram Bot Authorization Token
    bot = telegram.Bot('137079651:AAGb_XiKJ2m9gj-ODYBmQwz8uKrRSfZc_fI')

    # get the first pending update_id, this is so we can skip over it in case
    # we get an "Unauthorized" exception.
    try:
        update_id = bot.getUpdates()[0].update_id
    except IndexError:
        update_id = None

    logging.basicConfig(
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    while True:
        try:
            update_id = echo(bot, update_id)            
        except telegram.TelegramError as e:
            # These are network problems with Telegram.
            if e.message in ("Bad Gateway", "Timed out"):
                sleep(1)
            elif e.message == "Unauthorized":
                # The user has removed or blocked the bot.
                update_id += 1
            else:
                raise e
        except URLError as e:
            # These are network problems on our end.
            sleep(1)


def echo(bot, update_id):

    # Request updates after the last update_id
    for update in bot.getUpdates(offset=update_id, timeout=10):
        # chat_id is required to reply to any message
        chat_id = update.message.chat_id
        update_id = update.update_id + 1
        message = update.message.text
        if message:
            image_type = "Action"
            prefix = 'Here is your '            

            # you can change the query for the image  here
            query = message.encode('utf8')
            query= query.split()
            query='+'.join(query)
            url="https://www.google.co.in/search?q="+query+"&source=lnms&tbm=isch"
            header = {'User-Agent': 'Mozilla/5.0'}
            
            soup = get_soup(url,header)
            images = soup.find_all("img", {"src": re.compile("gstatic.com")}, limit=1)[0]['src']
            bot.sendMessage(chat_id=chat_id,
                            text=(prefix+message))
            bot.sendPhoto(chat_id=chat_id,photo=images)
            
    return update_id


def get_soup(url,header):
    return BeautifulSoup(urllib2.urlopen(urllib2.Request(url,headers=header)))

if __name__ == '__main__':
    main()